使用RNN 实现人脸图片分类

In [16]:
# 导入相关依赖
import torch
import torch.nn as nn
from charset_normalizer import models
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_olivetti_faces
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter


In [17]:
# 定义超参数
EPOCHS = 200
BATCH_SIZE = 32
LR = 0.001


In [29]:
# 加载数据
olivetti_faces = fetch_olivetti_faces(data_home="../dataset/olivettiFaces")
# 此处注意取 images 而非 data
data, target = olivetti_faces.images, olivetti_faces.target
# 划分训练集及测试集，并设置随机数种子，使用随机数种子确保代码每次运行时数据一致
# stratify=olivetti_faces.target 按照target 维度对数据集进行分层采样，却表40个人的数据都会被采集到
data_train, data_test, target_train, target_test = train_test_split(data, target, test_size=0.2, random_state=42, stratify=target)
# 将特征数据与目标值结合在一起
train_data_set = TensorDataset(torch.tensor(data_train, dtype=torch.float), torch.tensor(target_train, dtype=torch.long))
test_data_set = TensorDataset(torch.tensor(data_test, dtype=torch.float), torch.tensor(target_test, dtype=torch.long))
train_dataloader = DataLoader(train_data_set, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_data_set, batch_size=BATCH_SIZE, shuffle=True)



In [33]:
# 定义RNN 模型
model_list = {}
rnn_model = nn.RNN(
    # 输入特征数 64
    input_size=len(olivetti_faces.images[0]),
    # 隐藏状态特征数 128
    hidden_size=128,
    # 两层堆叠
    num_layers=2,
    # 设置第一维参数为批量（batch_size, sqe_len, feature）
    batch_first=True,
    # 添加偏置项
    bias=True
)

# 定义GRU 模型
gru_model = nn.GRU(
    input_size=len(olivetti_faces.images[0]),
    hidden_size=128,
    num_layers=2,
    batch_first=True,
    bias=True
)

# 定义 LSTM 模型
lstm_model = nn.LSTM(
    input_size=len(olivetti_faces.images[0]),
    hidden_size=128,
    num_layers=2,
    batch_first=True,
    bias=True
)

# 定义 BiRNN
biRnn_model = nn.RNN(
    input_size=len(olivetti_faces.images[0]),
    hidden_size=128,
    num_layers=2,
    batch_first=True,
    bias=True,
    # 双向RNN
    bidirectional=True,
)

# 定义 BiLSTM
biLstm_model = nn.LSTM(
    input_size=len(olivetti_faces.images[0]),
    hidden_size=128,
    num_layers=2,
    batch_first=True,
    bidirectional=True,
    bias=True
)
# 将模型放置到模型列表中
model_list.update({"rnn_model":rnn_model, "biRnn_model":biRnn_model, "biLstm_model":biLstm_model, "lstm_model":lstm_model, "gru_model":gru_model})


In [34]:
# 定义优化器及损失函数
def build_rnn_model(model_name):
    loss_fn = nn.CrossEntropyLoss()
    model = model_list.get(model_name)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    return loss_fn, model, optimizer

In [35]:

# 开始训练模型
writer = SummaryWriter()
# 定义输出线性层
liner = nn.Linear(128, 40)
for model_name in model_list.keys():
        # 初始化模型
        loss_fn, model, optimizer = build_rnn_model(model_name)
        for epoch in range(EPOCHS):
            model.train()
            for i, (images, labels) in enumerate(train_dataloader):
                optimizer.zero_grad()
                # 去除矩阵中值为1 的维度（颜色通道）
                outputs, h_l = model(images.squeeze())
                out = liner(outputs[:,-1,:])
                loss = loss_fn(out, labels)
                loss.backward()
                # 使用梯度裁剪，并设置最大L2范数为1.0， 超过这个值则会进行梯度裁剪
                torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm=1.0)
                optimizer.step()

                if i % 100 == 0:
                    print(f'Epoch [{epoch+1}/{EPOCHS}], {model_name} Loss: {loss.item():.4f}')
                    writer.add_scalar(f'{model_name} training loss', loss.item(), epoch * len(train_dataloader) + i)
            # 评估模型
            model.eval()
            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in test_dataloader:
                    outputs = model(images.squeeze())
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                accuracy = 100 * correct / total
                print(f'Epoch [{epoch+1}/{EPOCHS}], {model_name} Test Accuracy: {accuracy:.2f}%')
                writer.add_scalar(f'{model_name} test accuracy', accuracy, epoch)

Epoch [1/200], rnn_model Loss: 3.6830


AttributeError: 'tuple' object has no attribute 'data'